# Demo of the Periodic Hill case

The [phill example](https://github.com/KTH-Nek5000/KTH_Examples/tree/master/phill_STAT) has been adapted for a workflow using snek5000 and pymech. Here we will show how a workflow looks like. To get started, we install everything we need:

```sh
python -m venv venv
source venv/bin/activate
pip install snek5000 pymech -e "git+https://github.com/exabl/snek5000-phill#egg=phill"
```

```{note}
The repository `snek5000-phill` provides a Python package `phill` - the same
name as the `usr` files. More on packaging Nek5000 files can be found in the
{ref}`packaging tutorial <packaging>`.
```

## Initialize and setup up simulation paramters

In [1]:
from phill.solver import Simul

params = Simul.create_default_params()

INFO     

The following table matches counterpart of mandatory ``SIZE`` variables.

==========  ===================   =============================================
SIZE        params.oper           Comment
==========  ===================   =============================================
``ldim``    ``dim``               Domain dimensions (2 or 3)

``lpmin``   ``nproc_min``         Min MPI ranks
``lpmax``   ``nproc_max``         Max MPI ranks
``ldimt``   ``scalars``           Number of auxilary fields (minimum 1).

==========  ===================   =============================================


INFO     
The following table relate to element configuration for certain operations.
The parameters are considered "optional" (except for ``lx1`` / ``order`` and
``lxd`` / ``coef_dealiasing`` which are mandatory) and would be ignored with
the default values.

==========      ===================   =========================================
SIZE            params.oper.elem      Comment
==========  

The logger prints out some helpful information, which could be useful to a new user. It may also be suppressed by adjusting the log level as follows:

In [2]:
from snek5000.log import logger, logging

logger.setLevel(logging.ERROR)

The `params` object gives you a consolidated view of the parameters which are spread out in a typical Nek5000 case into `.par`, `.box` and `SIZE` file.  Already we seen that the parameters are more verbose, easier to understand. As a bonus, some parameters which depend on others are automatically set. For example, see {py:mod}`snek5000.operators`.

Now let us take a look at all the compilation parameters that we can modify. In a console the params would also output as follows:

In [3]:
print(params)


<params NEW_DIR_RESULTS="True" short_name_type_run="run">
  <oper Lx="1.0" Ly="1.0" Lz="1.0" boundary="['P', 'P', 'W', 'W', 'P', 'P']"
        dim="3" nproc_max="32" nproc_min="8" nx="22" ny="16" nz="19"
        origin_x="0.0" origin_y="0.0" origin_z="0.0" ratio_x="1.0" ratio_y="1.0"
        ratio_z="1.0" scalars="1">
    <max dim_krylov="30" dim_proj="20" hist="1" obj="1" order_time="3"
         perturb="1" scalars_cons="1" scalars_proj="1" sessions="1"/>  

    <elem coef_dealiasing="0.6666666666666666" order="6" order_out="6"
          staggered="True"/>  

    <misc fast_diag="False"/>  

  </oper>

  <output HAS_TO_SAVE="True" ONLINE_PLOT_OK="True" period_refresh_plots="1"
          sub_directory=""/>  

  <nek>
    <general dealiasing="True" dt="-0.0002" end_time="nan"
             extrapolation="standard" filter_cutoff_ratio="0.67"
             filter_weight="0.02" filtering="explicit" log_level="2"
             num_steps="20" opt_level="2" start_from="" stop_at="num_steps"
   

The parameters shown above can be modified too. For instance, let us tweak the number of elements, time-stepping and I/O parameters

In [4]:
# This affects both the box and SIZE files
params.oper.nx = 12
params.oper.ny = 10
params.oper.nz = 8

# This affects the par file 
params.nek.general.num_steps = 10
params.nek.general.time_stepper = "bdf2"
params.nek.general.write_interval = 10

params.nek.stat.av_step = 3
params.nek.stat.io_step = 10

Now initialize the simulation. This would copy the files based on the templates we have specified.

In [5]:
sim = Simul(params)
sim.path_run

PosixPath('/home/avmo/.local/scratch/tmp/phill_run_12x10x8_V1.x1.x1._2020-10-05_15-09-26')

In [6]:
!ls {sim.path_run}

etc		  params.xml  phill.par  SIZE	    templates
makefile_usr.inc  phill.box   phill.usr  Snakefile  toolbox


To run the simulation we need to execute certain commands. These are described using snakemake in the Snakefile. Let's look at the rules defined in the Snakefile (which are nearly generic for any Nek5000 case).

In [7]:
sim.make.list()

Changing to shadow directory: /home/avmo/.local/scratch/tmp/phill_run_12x10x8_V1.x1.x1._2020-10-05_15-09-26


test
all
mesh
compile
run
srun
clean
cleansimul
cleanall
archive
generate_box
move_box
generate_map
generate_makefile
generate_session


True

The rules in the Snakefile are either shell commands or Python code which handle different parts of the build step, such as building a mesh (rule `mesh`), compiling (rule `compile`) and running the simulation (rule `run` or `srun`). The rules can be executed on by one by passing them as strings to the `exec` method of the `sim.make` object. The default parameter is to do everyting to run a simulation.

In [9]:
sim.make.exec?

Signature: sim.make.exec(rules=('run',), dryrun=False)
Docstring:
Execute snakemake rules in sequence.

:returns: True if workflow execution was successful.
File:      ~/src/snek5000/snek5000/src/snek5000/make.py
Type:      method


In [10]:
sim.make.exec()

Changing to shadow directory: /home/avmo/.local/scratch/tmp/phill_run_12x10x8_V1.x1.x1._2020-10-05_15-09-26


Building DAG of jobs...
Executing subworkflow Nek5000.
Building DAG of jobs...
Nothing to be done.
Complete log: /home/avmo/src/snek5000/snek5000/lib/Nek5000/.snakemake/log/2020-10-05T151042.306490.snakemake.log
Executing main workflow.
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	compile
	1	generate_box
	1	generate_makefile
	1	generate_map
	1	generate_session
	1	move_box
	1	run
	7

[Mon Oct  5 15:10:42 2020]
rule generate_box:
    input: phill.box, /home/avmo/src/snek5000/snek5000/src/snek5000/../../lib/Nek5000/bin/genbox
    output: box.re2
    jobid: 5

[Mon Oct  5 15:10:42 2020]
Finished job 5.
1 of 7 steps (14%) done

[Mon Oct  5 15:10:42 2020]
rule move_box:
    input: box.re2
    output: phill.re2
    jobid: 1

[Mon Oct  5 15:10:42 2020]
Finished job 1.
2 of 7 steps (29%) done

[Mon Oct  5 15:10:42 2020]
rule generate_map:
    input: phill.re2, /home/avmo/src/snek5000/

True

In [14]:
!ls {sim.path_run}

build.log	  nek5000	 phill.log	   rs6phill0.f00002  templates
c2Dphill0.f00001  obj		 phill.ma2	   rs6phill0.f00003  toolbox
etc		  params.xml	 phill.par	   SESSION.NAME
logs		  phill0.f00001  phill.re2	   SIZE
makefile	  phill.box	 phill.usr	   Snakefile
makefile_usr.inc  phill.f	 rs6phill0.f00001  stsphill0.f00001


The simulation is done!